# Analyzing the stability of shear flows

In [1]:
import os.path
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
import trispectral as ts
from trispectral.stability import reduced_linear_operator

In [2]:
cwd = os.path.abspath("")

plt.style.use(os.path.join(cwd, "../misc/mpl_styles/main.mplstyle"))

## Wavevector

Consider a 3D domain with homogeneous $x$ and $z$ directions, such that

$f\left( x, y, z \right) = e^{i\alpha z} \hat{f}\left( x, y \right)$.

The Laplacian of $f$ then reads:

$\nabla^2 f = e^{i\alpha z} \left( \frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} - \alpha^2 \right)\hat{f}$.

In [3]:
alpha = 2.
wavevector = [0, None], [1, None], alpha

In [4]:
nx = ny = 41

grid = ts.Grid.from_bounds(
    [-1., 1., nx], [-1., 1., ny], discs=["chebyshev"] * 2
)

x, y = grid

In [5]:
fhat = 1. - x**2 - y**2

In [6]:
l = ts.scalar_laplacian_operator(grid, wavevector=wavevector) @ fhat

In [7]:
assert np.allclose(l, np.full_like(x, -4) - alpha**2 * fhat)

## Couette flow

In [3]:
alpha = 0.25
beta = 2.
wavevector = alpha, [0, None], beta

re = 800. # the Reynolds number

In [4]:
ny = 61

grid = ts.Grid.from_bounds([-1., 1., ny], discs=["chebyshev"])

y = grid[0]

In [5]:
flow = np.zeros(3 * ny)
flow[: ny] = y.copy()

In [6]:
mat = reduced_linear_operator(
    grid, flow, wavevector=wavevector, reynolds_number=re
)

In [7]:
ω = la.eigvals(mat)

ω *= 1j

ω = ω[ω.imag.argsort()[::-1]]
ω = ω[~np.isclose(ω, -1000j)]

In [8]:
print(ω[:10] / alpha)

[-6.53753740e-01-0.22021787j  6.53753740e-01-0.22021787j
 -3.99902941e-01-0.25329463j  3.99902941e-01-0.25329463j
  3.94622645e-01-0.36982719j -3.94622645e-01-0.36982719j
  3.72838533e-01-0.48343132j -3.72838533e-01-0.48343132j
  2.26052444e-10-0.4856143j  -1.82499726e-01-0.4924197j ]


## Pipe Poiseuille flow using 2D formulation

In [9]:
alpha = 1.
wavevector = [0, None], [1, None], alpha

re = 3000. # the Reynolds number

In [10]:
nphi, nr = 12, 41

grid = ts.Grid.polar(nphi, nr)

phi, r = grid[:, grid[1] > 0]

In [11]:
flow = np.zeros(3 * nphi * nr)
flow[2 * nphi * nr :] = 1 - r**2

In [12]:
mat = reduced_linear_operator(
    grid, flow, wavevector=wavevector, reynolds_number=re
)

In [13]:
ω = la.eigvals(mat)

ω *= 1j

ω = ω[ω.imag.argsort()[::-1]]
ω = ω[~np.isclose(ω, -1000j)]

In [14]:
print(ω[:15])

[0.91146557-0.04127564j 0.91146557-0.04127564j 0.94836022-0.05197311j
 0.9483602 -0.05197312j 0.88829766-0.06028569j 0.88829766-0.06028569j
 0.37093509-0.06161902j 0.37093509-0.06161902j 0.86436392-0.08325398j
 0.86436392-0.08325398j 0.35255493-0.08789898j 0.35255493-0.08789898j
 0.95820554-0.08834603j 0.95820554-0.08834603j 0.85478882-0.08887016j]


## Quasi-static MHD duct flow with axial magnetic field

In [41]:
alpha = 1.
wavevector = [0, None], [1, None], alpha

re = 1000. # the Reynolds number
ha = 10. # the Hartmann number

In [42]:
nx = ny = 41

grid = ts.Grid.from_bounds(
    [-1., 1., nx], [-1., 1., ny], discs=2 * ["chebyshev"]
)

x, y = grid

### Computing the Lorentz force

In [43]:
lhs = ts.scalar_laplacian_operator(grid, wavevector=wavevector)
rhs = ts.curl_operator(grid, wavevector=wavevector)[2 * nx * ny :]

In [44]:
bnds = grid.boundary_indices()
bnds = np.concatenate([np.concatenate(bnd) for bnd in bnds])

lhs[bnds] = np.identity(nx * ny)[bnds]
rhs[bnds] = 0.

In [45]:
f = la.solve(lhs, rhs)

In [46]:
f = -ha**2 / re * ts.gradient_operator(grid, wavevector=wavevector) @ f

f[: nx * ny, : nx * ny] -= ha**2 / re * np.identity(nx * ny)
f[
    nx * ny : 2 * nx * ny, nx * ny : 2 * nx * ny
] -= ha**2 / re * np.identity(nx * ny)

### Computing eigenvalues

In [47]:
flow = np.zeros(3 * nx * ny)
flow[2 * nx * ny :] = (1 - x**2) * (1 - y**2)

In [48]:
mat = reduced_linear_operator(
    grid,
    flow,
    wavevector=wavevector,
    reynolds_number=re,
    external_force_operator=f,
)

In [49]:
ω = la.eigvals(mat)

ω *= 1j

ω = ω[ω.imag.argsort()[::-1]]
ω = ω[~np.isclose(ω, -1000j)]

In [50]:
print(ω[0])

(0.9126383707938432-0.08902437171588663j)
